In [70]:
import json
import numpy as np

In [71]:
fs = open('songs_1.json','r')
s_json = json.load(fs)
fs.close()
print('there are {} songs'.format(len(s_json)))

there are 5835 songs


In [72]:
fc = open('chords.json','r')
c_json = json.load(fc)
fc.close()
print('there are {} chords'.format(len(c_json)))

there are 276 chords


In [81]:
chords = {}
chord_idx = 0

for c in c_json:
    name = c["name"]
    frets = c["frets"]
    fingers = c["fingers"]
    chords[name] = (frets,fingers)

In [82]:
songs = []
for s in s_json:
    songs.append(s)

In [83]:
def chord_transition_cost(c0, c1, verbose=False):
    '''
    So we want to look at each finger, see which fret it started and ended on, and take the manhattan distance
    of that
    Add up all of those
    
    fing[i] represents which finger should be on string i
    fret[i] represents which fret should be pressed on string i
    
    we look at the nonzero fret values, and see the associated finger and string values
    and from this we can construct a map from fingers to string and fret
    
    but how to deal with barring(I think we should just take the first fret that we find a finger for)
    '''
    start = [None, None, None, None]
    end = [None, None, None, None]
    
    for string in range(4):
        start_fret = int(c0[0][string])
        end_fret = int(c1[0][string])
        start_finger = int(c0[1][string])
        end_finger = int(c1[1][string])
        if(start_finger != 0 and start[start_finger-1] is None):
            start[start_finger-1] = (string, start_fret)
        if(end_finger != 0 and end[end_finger-1] is None):
            end[end_finger-1] = (string, end_fret)
    if verbose:
        print(start)
        print(end)
    
    cost = 0
    for i in range(4):
        if(end[i] is None):
            cost += 0 #removing a finger (or potentially not using a finger)
        elif(start[i] is None):
            cost += end[i][1] #the cost is just the fret that you place the finger on
        else:
            cost += abs(start[i][0] - end[i][0]) + abs(start[i][1] - end[i][1])
            #manhattan distance between start and end frets/strings
    
    return cost

def ctc(c0, c1):
    return chord_transition_cost(chords[c0], chords[c1])

In [84]:
def chord_lev(chord1, chord2, verbose=True):
    # there's one less chord transition than there are chords
    l1 = len(chord1) 
    l2 = len(chord2)
    dp = np.zeros((l1,l2))
    for i in range(l1-1):
        dp[i+1][0] = dp[i][0] + ctc(chord1[i], chord1[i+1])
    for j in range(l2-1):
        dp[0][j+1] = dp[0][j] + ctc(chord2[j], chord2[j+1])
    
    for i in range(1, l1):
        for j in range(1,l2):
            t1 = (chord1[i-1], chord1[i]) #transtion a
            t2 = (chord2[j-1], chord2[j]) #transition b
            val = 0
            if t1 == t2: #if the chords are equal, then the cost 
                val = dp[i-1][j-1]
            else:
                '''
                chords 1: a1, a2, a3... an
                chords 2: b1, b2, b3... bm
                
                replacing ai with bj:
                    original: ai-1 -> ai -> ai+1
                    new: ai-1 -> bj -> ai+1
                    cost: (ai-1->bj)
                deleting ai:
                    original: ai-1 -> ai -> ai+1
                    new: ai-1 -> ai+1
                    cost: (ai-1->ai+1) - (ai-1->i) - (ai->ai+1)
                inserting bj after ai:
                    original: ai-1 -> ai -> ai+1
                    new: ai-1 -> ai -> bj -> ai+1
                    cost: (ai->bj) + (bj->ai+1) - (ai->ai+1)
                '''
                '''
                replace at with bt is going to be +bt-at
                delete at = -at
                insert bt = +bt
                '''
                #or something like that
                replace = dp[i-1][j-1] - ctc(t1[0], t1[1]) + ctc(t2[0],t2[1])
                delete = dp[i-1][j] - ctc(t1[0],t1[1])
                insert = dp[i][j-1] + ctc(t2[0],t2[1])
                val = min(replace, delete, insert)
            dp[i][j] = val
    if verbose:
        print(dp)
    return(dp[l1-1][l2-1])

In [85]:
def song_diff(song1, song2, verbose=True):
    c1 = song1['chords']
    c2 = song2['chords']
    return chord_lev(c1,c2, verbose=verbose)

In [88]:
l = 10 #len(songs)
diffs = np.zeros((l,l))
for i in range(l):
    for j in range(i,l):
        diffs[i][j] = song_diff(songs[i], songs[j], verbose=False)
        print(i,j,diffs[i][j])

0 0 0.0
0 1 -314.0
0 2 -156.0
0 3 -227.0
0 4 -304.0
0 5 175.0
0 6 -228.0
0 7 67.0
0 8 -263.0
0 9 48.0
1 1 0.0
1 2 158.0
1 3 87.0
1 4 10.0
1 5 489.0
1 6 86.0
1 7 381.0
1 8 51.0
1 9 362.0
2 2 0.0
2 3 -71.0
2 4 -148.0
2 5 331.0
2 6 -72.0
2 7 223.0
2 8 -107.0
2 9 204.0
3 3 0.0
3 4 -77.0
3 5 402.0
3 6 -1.0
3 7 294.0
3 8 -36.0
3 9 275.0
4 4 0.0
4 5 479.0
4 6 76.0
4 7 371.0
4 8 41.0
4 9 352.0
5 5 0.0
5 6 -403.0
5 7 -108.0
5 8 -438.0
5 9 -127.0
6 6 0.0
6 7 295.0
6 8 -35.0
6 9 276.0
7 7 0.0
7 8 -330.0
7 9 -19.0
8 8 0.0
8 9 311.0
9 9 0.0


In [89]:
song_diff(songs[10], songs[8])

[[   0.    1.    4. ...  201.  204.  209.]
 [   5.   -4.   -1. ...  196.  199.  204.]
 [  12.  -11.   -8. ...  189.  192.  197.]
 ...
 [ 200. -199. -196. ...    1.    4.    9.]
 [ 207. -206. -203. ...   -6.   -3.    2.]
 [ 210. -209. -206. ...   -9.   -6.   -1.]]


-1.0

In [92]:
print(songs[10])

{'artist': '4-non-blondes', 'song': 'whats-up', 'chords': ['A', 'Bm', 'D', 'A', 'A', 'Bm', 'D', 'A', 'A', 'Bm', 'D', 'A', 'A', 'Bm', 'D', 'A', 'A', 'Bm', 'D', 'A', 'A', 'Bm', 'D', 'A', 'A', 'Bm', 'D', 'A', 'A', 'Bm', 'D', 'A', 'A', 'Bm', 'D', 'A', 'A', 'Bm', 'D', 'A', 'A', 'Bm', 'D', 'A', 'A', 'Bm', 'D', 'A', 'A', 'Bm', 'D', 'A', 'A', 'Bm', 'D', 'A'], 'difficulty': 2}


In [93]:
print(songs[8])

{'artist': '3oh3', 'song': 'dont-trust-me', 'chords': ['Am', 'F', 'C', 'G', 'Am', 'F', 'C', 'G', 'Am', 'F', 'C', 'G', 'Am', 'F', 'C', 'G', 'Am', 'F', 'C', 'G', 'Am', 'F', 'C', 'G', 'Am', 'F', 'C', 'G', 'Am', 'F', 'C', 'G', 'Am', 'F', 'B', 'B', 'B', 'C', 'G', 'Am', 'F', 'C', 'G', 'Am', 'F', 'C', 'G', 'Am', 'F', 'C', 'G', 'Am', 'F', 'C', 'G', 'Am', 'F', 'C', 'G', 'Am', 'F', 'C', 'G', 'Am', 'F', 'C', 'G', 'Am', 'F', 'C', 'G'], 'difficulty': 1}


In [97]:
song_graph = np.genfromtxt('diffs.csv', delimiter=',')

KeyboardInterrupt: 